# TripScore — Pipeline Walkthrough (Offline / Stubbed)\n
\n
This notebook runs the full recommendation pipeline **without network access** by injecting stub clients.\n
\n
Goal:\n
- show how ingestion outputs feed features,\n
- how feature scores become an explainable composite breakdown,\n
- how ranking works end-to-end.\n

In [1]:
import sys
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / "src").exists() and (p / "src" / "tripscore").exists():
            return p
        if (p / ".git").exists():
            return p
        if (p / ".env").exists():
            return p
        p = p.parent
    return start.resolve()

repo_root = find_repo_root(Path.cwd())
src_dir = repo_root / "src"
if src_dir.exists():
    sys.path.insert(0, str(src_dir))

In [2]:
from datetime import datetime
from zoneinfo import ZoneInfo

from tripscore.config.settings import get_settings
from tripscore.domain.models import GeoPoint, TimeWindow, UserPreferences
from tripscore.ingestion.tdx_client import (
    BikeStationStatus,
    BusStop,
    MetroStation,
    ParkingLotStatus,
)
from tripscore.ingestion.weather_client import WeatherSummary
from tripscore.recommender.recommend import recommend

settings = get_settings()
tz = ZoneInfo(settings.app.timezone)

prefs = UserPreferences(
    origin=GeoPoint(lat=25.0478, lon=121.5170),
    time_window=TimeWindow(
        start=datetime(2026, 1, 5, 10, 0, tzinfo=tz),
        end=datetime(2026, 1, 5, 18, 0, tzinfo=tz),
    ),
    # Optional: tag preferences can be set here\n
    tag_weights={'food': 1.0, 'culture': 0.6, 'indoor': 0.3},
    max_results=5,
)

## Stub clients\n
\n
The recommender accepts dependency injection: you can pass `tdx_client=` and `weather_client=`.\n
Here we provide deterministic stub data so the notebook is fully offline.\n

In [3]:
class StubTdxClient:
    def get_bus_stops(self, *, city=None):
        return [
            BusStop(stop_uid='B1', name='Stub Bus Stop A', lat=25.0470, lon=121.5170),
            BusStop(stop_uid='B2', name='Stub Bus Stop B', lat=25.0500, lon=121.5200),
        ]

    def get_youbike_station_statuses(self, *, city=None):
        return [
            BikeStationStatus(
                station_uid='Y1',
                name='Stub YouBike Station',
                lat=25.0480,
                lon=121.5165,
                available_rent_bikes=12,
                available_return_bikes=18,
            )
        ]

    def get_metro_stations(self, *, operators=None):
        return [
            MetroStation(station_uid='M1', name='Stub Metro Station', lat=25.0475, lon=121.5175, operator='TRTC')
        ]

    def get_parking_lot_statuses(self, *, city=None):
        return [
            ParkingLotStatus(
                parking_lot_uid='P1',
                name='Stub Parking Lot',
                lat=25.0468,
                lon=121.5172,
                available_spaces=120,
                total_spaces=300,
            )
        ]


class StubWeatherClient:
    def get_summary(self, *, lat: float, lon: float, start, end):
        # A mild, "good weather" window.\n
        return WeatherSummary(max_precipitation_probability=10.0, mean_temperature_c=25.0)

In [4]:
result = recommend(
    prefs,
    settings=settings,
    tdx_client=StubTdxClient(),
    weather_client=StubWeatherClient(),
) 

print('Generated at:', result.generated_at)
print('Results:', len(result.results))
print('Top 3 names:', [r.destination.name for r in result.results[:3]])

Generated at: 2026-01-18 13:21:15.785726+08:00
Results: 5
Top 3 names: ['National Taiwan Museum', 'Lin An Tai Historical House and Museum', 'Shuanglian Market']


In [5]:
from pprint import pprint

# Show one destination's explainable breakdown\n
item = result.results[0]
print('Destination:', item.destination.name)
print('Total:', item.breakdown.total_score)
for comp in item.breakdown.components:
    print('-', comp.name, 'score=', round(comp.score, 3), 'weight=', round(comp.weight, 2))
    print('  reasons:', '; '.join(comp.reasons[:4]))
    # Uncomment to inspect full details payload\n
    # pprint(comp.details)\n

Destination: National Taiwan Museum
Total: 0.5517140390255331
- accessibility score= 0.363 weight= 0.35
  reasons: ~0.6 km from origin; 0 bus stops within 500m; Nearest bus stop ~547m; 1 metro stations within 700m
- weather score= 0.938 weight= 0.3
  reasons: Max rain probability 10%; Avg temperature 25.0°C; Rain impact adjusted for indoor/outdoor
- preference score= 0.419 weight= 0.2
  reasons: Matches: culture, family_friendly, indoor
- context score= 0.396 weight= 0.15
  reasons: Predicted crowd risk high (0.80); Parking availability suggests high congestion; Family-friendly destination
